## Extra credit challenge

You have bunch of empty boxes. To store them efficiently you are planning to place them inside each other. Each box has three dimensions `(h, w, l)`, assume that you can place box A inside box B if all of its dimensions are smaller, i.e., `hA<hB`, `wA<wB`, and `lA<lB`. (You don't like to rotate your boxes!) Write a function that
* Takes a list of tuples as input, where each tuple describes the dimensions of a box. No two boxes have exactly the same size.
* Returns the longest sequence of boxes that you can place inside of each other in a nested fashion.

Example

Input:
```
[(10,10,5),(1,1,1),(2,2,8),(9,5,3)]
```
Output:
```
[(10,10,5),(9,5,3),(1,1,1)]
```

Create a graph, each node is a box with properties (h,w,l).

If a box can be put into another box, then there is a directed edge from the lower box to the higher box. <br>
Therefore we can construct the edges in the graph this way: for v in vertices vs for u in vertices, if u can be put in v or v in u, then add edge (u,v) or (v,u) to the graph.

The graph is DAG, most simply proved: if there is a directed cycle, then there'd a box that can be put on itself. Because of this, we can tackle the problem two ways:<br>
a) do a topological sort, then find the longest path<br>
b) Set edge weights to -1 and find the smallest value path (shortest path). This'd actually result in biggest length, as the sum of weights would be len*(-1)<br>
(The way I did such things in my BSc thesis.)

I chose the second approach.

In [1]:
import numpy as np
import networkx as nx
# Create a graph, each node is a box with (h,w,l), 

# if a box can be put on another box, then there is a directed edge from the lower box to the higher box. Therefore: for v in vertices vs for u in vertices, if u can be put on v or v in u, then add edge (u,v) to the graph.

# The graph is DAG, most simply said if there is a directed cycle, then there'd a box that can be put on itself. Then we can make do two things:
# a) topological sort, then find the longest path (This was suggested by copilot)
# b) Set edge weights to -1 and find the shortest path. This'd actually result in biggest length, as the sum of weights would be len*(-1)


# B)
def box_graph(boxes):
    G = nx.DiGraph()
    for i in range(len(boxes)):
        G.add_node(i, height=boxes[i][0], width=boxes[i][1], length=boxes[i][2])
    for i in range(len(boxes)):
        for j in range(i+1,len(boxes)):
            if G.nodes[i]['height'] > G.nodes[j]['height'] and G.nodes[i]['width'] > G.nodes[j]['width'] and G.nodes[i]['length'] > G.nodes[j]['length']:
                G.add_edge(i,j,weight=-1)
            elif G.nodes[i]['height'] < G.nodes[j]['height'] and G.nodes[i]['width'] < G.nodes[j]['width'] and G.nodes[i]['length'] < G.nodes[j]['length']:
                G.add_edge(j,i,weight=-1)
    #Finished creating the graph
    paths = nx.shortest_path(G,weight='weight', method='bellman-ford')
    #This returns a dict of paths (dict), path[source][target] returns the shortest path from source to target. So I iterate over the dicts and gradually set the maximum
    source=0; target=0; path=[] #Not sure about initalization
    for iter in paths:
        for iter2 in paths[iter]:
            if len(paths[iter][iter2]) > len(path):
                path = paths[iter][iter2]
                source = iter
                target = iter2
    path_boxes = []
    for i in path:
        path_boxes.append(boxes[i])
    
    return path_boxes
    

In [2]:
path_boxes= box_graph([(10,10,5),(1,1,1),(2,2,8),(9,5,3)])
print(path_boxes);



[(10, 10, 5), (9, 5, 3), (1, 1, 1)]
